## Проект: Анализ популярных в Spotify песен по странам

Мы взяли проект по анализу данных о песнях в сервисе Spotify, чтобы выяснить как отличаются вкусы у слушателей из разных стран, и какие песни желает аудитория в разных странах

![Spotify](image.png)


Для начала включим библиотеки, которые понадобятся нам для работы

In [202]:
import numpy
import pandas as pd
import plotly as pl 
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

## 1. Работа с данными

Для анализа мы будем использовать два csv файла:
1) Все песни Spotify
2) Топ популярных песен по странам на сегодняшний день

Мы используем два файла, т.к. в таблице популярных песен отсутствует жанр. Ниже мы сливаем две таблицы в одну где будет как жанр, так и все остальные нужные данные. Мерджим по айдишнику, так как он уникален для всех песен

In [203]:
allSongs = pd.read_csv("All spoty songs.csv")
uniTop = pd.read_csv("universal_top_spotify_songs.csv")

allSongs1 = allSongs[['track_genre', 'track_id']]
df = pd.merge(allSongs1, uniTop, left_on="track_id", right_on = "spotify_id", how="inner")

In [205]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 0 to 14517
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   track_genre         14518 non-null  object 
 1   track_id            14518 non-null  object 
 2   spotify_id          14518 non-null  object 
 3   name                14518 non-null  object 
 4   artists             14518 non-null  object 
 5   daily_rank          14518 non-null  int64  
 6   daily_movement      14518 non-null  int64  
 7   weekly_movement     14518 non-null  int64  
 8   country             14228 non-null  object 
 9   snapshot_date       14518 non-null  object 
 10  popularity          14518 non-null  int64  
 11  is_explicit         14518 non-null  bool   
 12  duration_ms         14518 non-null  int64  
 13  album_name          14515 non-null  object 
 14  album_release_date  14515 non-null  object 
 15  danceability        14518 non-null  float64
 16  ener

На выходе получаем датасет со следующими полями, где большая часть налл значений приходится на страны. Значение null в стране значит, что песня попала в топ по миру, поэтому в заполнении данные ячейки не нуждаются.

Также, нам для исследования не пригодятся столбцы time_signature, mode, key, snapshot_date, daily_rank, album_name,  daily_movement, weekly_movement, album_release_date и spotify_id, поэтому мы смело можем от них избавиться

In [211]:
columns_to_delete = ['time_signature', 'mode', 'key', 'snapshot_date', 'daily_rank', 'daily_movement', 'weekly_movement', 'spotify_id', 'album_release_date', 'album_name']
df = df.drop(columns=columns_to_delete, errors='ignore')

df.head(5)

,track_genre,track_id,name,artists,country,popularity,is_explicit,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,alt-rock,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,NaN,92,False,240400,0.612,0.807,-2.81,0.0336,0.0495,0.0177,0.101,0.398,124.053
1,alt-rock,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,US,92,False,240400,0.612,0.807,-2.81,0.0336,0.0495,0.0177,0.101,0.398,124.053
2,alt-rock,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,UA,92,False,240400,0.612,0.807,-2.81,0.0336,0.0495,0.0177,0.101,0.398,124.053
3,alt-rock,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,SK,92,False,240400,0.612,0.807,-2.81,0.0336,0.0495,0.0177,0.101,0.398,124.053
4,alt-rock,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,The Neighbourhood,SA,92,False,240400,0.612,0.807,-2.81,0.0336,0.0495,0.0177,0.101,0.398,124.053


Получаем удобный для чтения и работы датасет, поэтому приступим к анализу

## 2. Проведение анализа

### 1. Проверим популярность жанров во всем мире
Для начала, узнаем какие жанры в музыке являются самыми популярными во всем мире. Для этого, сделаем словарь словарей, где ключ внешнего словаря - страна, ключ внутреннего - жанр, значение внутреннего - подсчет популярных песен в этом жанре строго внутри нужной страны. А после, визуализируем полученные данные удобным пирогом

In [214]:
dictCounriesGenres = {}


# Проходимся по каждой уникальной стране датасета,
# берем уникальные треки и считаем количество треков в каждом жанре внутри страны
for country in df['country'].unique():
    country_df = df[df['country'] == country]
    unique_tracks = country_df.drop_duplicates(subset='track_id')
    genre_count = unique_tracks['track_genre'].value_counts().to_dict()
    dictCounriesGenres[country] = genre_count

    
#     Разкомментить для проверки:
# for key, val in dictCounriesGenres.items():
#     print(key, ':', val)

Теперь, возьмем топ 2 самых популярных жанра в каждой стране, и сделаем новый словарь, в котором ключами будут служить музыкальные жанры, а значениями - списки стран, в которых этот жанр входит в топ 2 по популярности

In [123]:
top_genres_dict = {}


for country, genre_count in result_dict.items():
    sorted_genres = sorted(genre_count.items(), key=lambda x: x[1], reverse=True)
    top_genres = [genre for genre, count in sorted_genres[:2]]
    
    for genre in top_genres:
        if genre not in top_genres_dict:
            top_genres_dict[genre] = [country]
        else:
            top_genres_dict[genre].append(country)

for k, v in top_genres_dict.items():
    print(k, ':', v)

country : ['US', 'CA', 'NZ']
disco : ['US', 'CA', 'AT', 'IE']
garage : ['UA', 'PL', 'LT', 'KZ', 'EE', 'IE', 'PT', 'ID', 'BG']
alt-rock : ['UA', 'SK', 'PL', 'KZ', 'GB', 'EE', 'CZ', 'AE', 'CR', 'AU']
chill : ['SK', 'CZ', 'MY', 'SE', 'NO', 'NL', 'LU', 'FR', 'DE', 'BE', 'IS', 'PT', 'IL']
ambient : ['SA']
pop : ['SA', 'LV', 'GB', 'CH', 'AE', 'AU', 'SG', 'MY', 'LU', 'BE', 'PK', 'ID', 'BG', 'ZA', 'PH']
indie-pop : ['LV', 'LT']
dance : ['CH', 'AT', 'NO', 'DE', 'IL', 'TW', 'KR']
latino : ['CR', 'HN', 'SV', 'PE', 'PA', 'NI', 'GT', 'EC', 'BO', 'VE', 'PY', 'CO', 'ES']
folk : ['NZ', 'IS']
j-pop : ['JP']
anime : ['JP']
cantopop : ['HK']
mandopop : ['SG', 'TW']
french : ['FR']
hip-hop : ['PK', 'IN', 'ZA', 'IT']
emo : ['BY']
punk-rock : ['BY']
indian : ['IN']
k-pop : ['TH']
latin : ['HN', 'SV', 'PE', 'PA', 'NI', 'GT', 'EC', 'DO', 'BO', 'VE', 'PY', 'CO']
rock-n-roll : ['DK']
turkish : ['TR']


Получаем красивый словарь, по которому уже видны "жанровые лидеры", но, для удобства, визуализируем полученные данные

In [124]:
counts = [len(countries) for countries in top_genres_dict.values()]
genres = list(top_genres_dict.keys())

pie_data = pd.DataFrame({'Genre': genres, 'Count': counts})

fig = px.pie(pie_data, values='Count', names='Genre', title='Популярность жанров в мире')
fig.show()

Как мы видим, жанр latin(+ latino) то есть латинская музыка, является самым популярным в мире. На втором и третьем местах идут Поп и Чилл, т. е. музыка для расслабления, которую слушают все возраста

### 2. Сравнение числовых параметров песен по странам
С жанрами, что является очень общей оценкой, разобрались, теперь стоит посмотреть на математические значения.
Нашей целью будет посмотреть на средние числовые значения у популярных песен по странам, и предоставить четкую визуализацию, чтобы даже визуально можно будет определить в каких странах слушатели любят танцевальную и быструю музыку, а в каких предпочитают плавную и инструментальную.

Для этого, мы создадим новый dataFrame с нужными для нас полями, и, проходясь по странам, найдем средние значения энергичности, громкости, танцевальности, темпа, акустичности, насыщенности текстом и инструментальности.

In [216]:
dfAvg = pd.DataFrame(columns=['country', 
                              'avg_duration', 'avg_danceability', 'avg_energy', 
                              'avg_speechiness', 'avg_loudness', 'avg_acousticness', 
                              'avg_instrumentalness', 'avg_liveness', 'avg_tempo'])

dfs_to_concat = []

# Итерируемся по уникальным странам
for country in df['country'].unique():
    #Берем только строки, которые относятся к нашей стране
    country_df = df[df['country'] == country]
    # Удаляем дубликаты треков, если такие есть 
    unique_tracks = country_df.drop_duplicates(subset='track_id')
    
#   Создаем массив для добавления в новый датаФрейм
    newRow = pd.DataFrame({
        'country': [country],
        'avg_duration': [country_df['duration_ms'].mean()],
        'avg_danceability': [country_df['danceability'].mean()],
        'avg_energy': [country_df['energy'].mean()],
        'avg_speechiness': [country_df['speechiness'].mean()],
        'avg_loudness': [country_df['loudness'].mean()],
        'avg_acousticness': [country_df['acousticness'].mean()],
        'avg_instrumentalness': [country_df['instrumentalness'].mean()],
        'avg_liveness': [country_df['liveness'].mean()],
        'avg_tempo': [country_df['tempo'].mean()]
    })
    dfs_to_concat.append(newRow)

# Новым (Теперь append заменяют на concat) методом заполняем нашу базу данных
dfAvg = pd.concat(dfs_to_concat, ignore_index=True)
df = df.drop(df.index[0])
    
dfAvg.head(10)

,country,avg_duration,avg_danceability,avg_energy,avg_speechiness,avg_loudness,avg_acousticness,avg_instrumentalness,avg_liveness,avg_tempo
0,US,231538.276699,0.570257,0.641262,0.087966,-5.901500,0.238039,0.007848,0.132735,136.289350
1,UA,206798.479638,0.570624,0.635389,0.031189,-5.802683,0.081008,0.016395,0.100169,98.115041
2,SK,220390.309677,0.534884,0.761439,0.036143,-5.161258,0.267686,0.005490,0.187454,124.900955
3,SA,236645.349823,0.493419,0.587883,0.045339,-7.359912,0.279733,0.216884,0.112214,111.442147
4,PL,215643.859649,0.547088,0.635947,0.030091,-5.676228,0.087439,0.019586,0.099421,99.261333
5,LV,211738.581272,0.557477,0.627913,0.067744,-6.589737,0.184969,0.056615,0.142590,113.922023
6,LT,215095.341686,0.547032,0.625415,0.040833,-6.435337,0.109507,0.102980,0.130552,105.096699
7,KZ,217934.237668,0.564363,0.633457,0.059935,-5.780740,0.093161,0.017177,0.104003,110.261744
8,GB,210315.484375,0.501383,0.678703,0.051215,-6.496755,0.202911,0.006717,0.130739,119.299154
9,EE,217367.773234,0.553305,0.647810,0.056165,-6.015015,0.181583,0.012481,0.120847,114.520186


Получаем красивую табличку, по которой теперь можно визуализировать данные. 
Мы посчитали уместным построить несколько barchart диаграмм и поставить их рядом, 
т.к. range значений у каждого параметра разный.

Для построения диаграмм используем plotly subplots и plotly graph objects. Указываем название для каждой диаграмки. 
Подписывать оси и указывать легенду здесь не уместно, т.к. нас интересуют не пиковые значения, а страны с пиковыми значениями

In [221]:
fig = sp.make_subplots(rows=2, cols=2, subplot_titles = ['Продолжительность', 'Энергичность', 'Инструментальность', 'Громкость'])

fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_duration'], name='Продолжительность'), row=1, col=1)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_energy'], name='Энергичность'), row=1, col=2)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_instrumentalness'], name='Инструментальность'), row=2, col=1)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_loudness'], name='Громкость'), row=2, col=2)

fig.update_layout(title_text='Сравнение стран по средним значениям №1', showlegend=False)

fig.show()

По первой группе диаграмм видим, что седняя предпочитаемая продолжительность сильно не отличается, выделить можно только $Исландию$ и $Корею$, в которых предпочитают короткие треки, и $ЮАР$, который лидирует по максимальной средней длительности

А вот по остальным трем параметрам можно четко выделить несколько пиковых значений.
Для инструментальности, явный фаворит - $Саудовская$ $Аравия$, которая в два раза превышает второе место $Литву$.
В энергичности и громкости, сильно выделяется снова $Исландия$, где любят тихую и неЭнергичную музыку, а так же $Гонконг$, где так же предпочитают менее энергичную музыку, и $Турция$, которая занимает второе место по минимальной средней громкости

Перейдем ко второй группе диаграмм.

In [219]:
fig = sp.make_subplots(rows=2, cols=2, subplot_titles = ['Темп', 'Насыщенность словами', 'Акустичность', 'Танцевальность'])

fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_tempo'], name='Продолжительность'), row=1, col=1)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_speechiness']), row=1, col=2)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_acousticness']), row=2, col=1)
fig.add_trace(go.Bar(x=dfAvg['country'], y=dfAvg['avg_danceability']), row=2, col=2)

fig.update_layout(title_text='Сравнение стран по средним значениям №2', showlegend=False)

fig.show()

Здесь картина сильно меняется. Абсолютные любители быстрой музыки - $Испания$ и $Дания$, а медленной - $Израиль$.
$Испания$ так же занимает лидерство в музыке с большим количеством слов. Второе и третье место держат $Болгария$ и $ЮАР$.
Самая акустичная страна - $Исландия$, что кореллирует с Инструментальностью, а вот $Тайланд$ и $Италия$ совсем такую музыку не любят
С Танцевальностью всё так не однозначно. На первое место вырывается $Корея$, но дальше места делят большое количество стран. Последние строчки занимают $Беларусь$ и $Филиппины$

 ### 3. Анализ стран-любителей грубых песен
 Далее проведем анализ песен с присутствием ненормативной (explicit) лексики. Для этого разделим наш dataFrame на два, с присутствием и отсутствием таковой.
 Для визуализации мы подобрали двойную skatter диаграмму, которая покажет рейтинг песен по странам (так как это топ, почти весь рейтинг будет высоким, это можно заметить конентрированием точек наверху), и разделит все песни на ругательные и нет.

In [231]:
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=['Explicit', 'Non-Explicit'])

# Explicit
explicit_df = df[df['is_explicit'] == True]
explicit_df = explicit_df.sort_values(['country', 'popularity'])
fig.add_trace(go.Scatter(x=explicit_df['country'], y=explicit_df['popularity'], mode='markers', name='Explicit'), row=1, col=1)

# Non-Explicit
non_explicit_df = df[df['is_explicit'] == False]
non_explicit_df = non_explicit_df.sort_values(['country', 'popularity'])
fig.add_trace(go.Scatter(x=non_explicit_df['country'], y=non_explicit_df['popularity'], mode='markers', name='Non-Explicit'), row=1, col=2)


#  Добавляем название и подписи осям, убираем легенду т.к. она здесь не уместна
fig.update_layout(title_text='Сравнение рейтинга ругательных песен по странам', showlegend=False)
fig.update_xaxes(title_text='Страна', row=1, col=1)
fig.update_xaxes(title_text='Страна', row=1, col=2)
fig.update_yaxes(title_text='Рейтинг', row=1, col=1)
fig.update_yaxes(title_text='Рейтинг', row=1, col=2)


fig.show()

Также сделаем совместную диаграмму, для наглядности

In [232]:
fig = sp.make_subplots(rows=1, cols=1)

df = df.sort_values(['country', 'popularity'])

explicit_df = df[df['is_explicit'] == True]
fig.add_trace(go.Scatter(x=explicit_df['country'], y=explicit_df['popularity'], mode='markers', name='Explicit'))

non_explicit_df = df[df['is_explicit'] == False]
fig.add_trace(go.Scatter(x=non_explicit_df['country'], y=non_explicit_df['popularity'], mode='markers', name='Non-Explicit'))

fig.update_layout(title_text='Совместная диаграма по норм. лексике', showlegend=True)
fig.update_xaxes(title_text='Страна', row=1, col=1)
fig.update_yaxes(title_text='Рейтинг', row=1, col=1)

fig.show()

Можем заметить, что в топе находится намного больше (примерно в два раза), песен без ненормативной лексики, а во многих странах, таких как $Япония$, $Украина$, $Гонконг$, $Индонезия$, $Сингапур$, в топе нет ни одной песни с присутствием ненормативной лексики.

Лидером в количестве топ песен с ненормативной лексикой является $Латвия$